# 23장 훈련 모델의 저장, 로딩, 서빙

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-python-cookbook-2nd/blob/main/ch23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [ ]:
import numpy as np
import sklearn
import joblib
import tensorflow
import torch
import flask

print('numpy', np.__version__)
print('sklearn', sklearn.__version__)
print('joblib', joblib.__version__)
print('tensorflow', tensorflow.__version__)
print('torch', torch.__version__)
print('flask', flask.__version__)

numpy 1.25.2
sklearn 1.2.2
joblib 1.3.2
tensorflow 2.15.0
torch 2.1.0+cu121
flask 2.2.5


## 23.1 사이킷런 모델 저장하고 로드하기

In [ ]:
# 라이브러리를 임포트합니다.
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 결정 트리 분류기 객체를 만듭니다.
classifer = RandomForestClassifier()

# 모델을 훈련합니다.
model = classifer.fit(features, target)

# 모델을 피클 파일로 저장합니다.
joblib.dump(model, "model.pkl")

['model.pkl']

In [ ]:
# 파일에서 모델을 복원합니다.
classifer = joblib.load("model.pkl")

In [ ]:
# 새로운 샘플을 만듭니다.
new_observation = [[ 5.2,  3.2,  1.1,  0.1]]

# 샘플의 클래스를 예측합니다.
classifer.predict(new_observation)

array([0])

In [ ]:
# 라이브러리를 임포트합니다.
import sklearn

# 사이킷런 버전을 구합니다.
scikit_version = sklearn.__version__

# 모델을 피클 파일로 저장합니다.
joblib.dump(model, "model_{version}.pkl".format(version=scikit_version))

['model_1.2.2.pkl']

## 23.2 텐서플로 모델 저장하고 로드하기

In [ ]:
# 라이브러리를 임포트합니다.
import numpy as np
from tensorflow import keras

# 랜덤 시드를 지정합니다.
np.random.seed(0)

# 하나의 은닉층을 가진 모델을 만듭니다.
input_layer = keras.Input(shape=(10,))
hidden_layer = keras.layers.Dense(10)(input_layer)
output_layer = keras.layers.Dense(1)(input_layer)
model = keras.Model(input_layer, output_layer)
model.compile(optimizer="adam", loss="mean_squared_error")

# 모델을 훈련합니다.
x_train = np.random.random((1000, 10))
y_train = np.random.random((1000, 1))
model.fit(x_train, y_train)

# `save_model`이란 디렉토리에 모델을 저장합니다.
model.save("saved_model")

32/32 [==============================] - 2s 5ms/step - loss: 0.1992


In [ ]:
!ls saved_model

assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


## 23.3 파이토치 모델 저장하고 로드하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Dropout(0.1), # 10%의 뉴런을 드롭아웃합니다.
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 5
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# 모델을 훈련한 다음 저장합니다.
torch.save(
    {
        'epoch': epoch,
        'model_state_dict': network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    },
    "model.pt"
)

# 신경망을 로드합니다.
network = SimpleNeuralNet()
state_dict = torch.load(
    "model.pt",
    map_location=torch.device('cpu')
    )["model_state_dict"]
network.load_state_dict(state_dict, strict=False)
network.eval()

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


SimpleNeuralNet(
  (sequential): Sequential(
    (0): Linear(in_features=10, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): Sigmoid()
  )
)